## SQL kódy pro zodpovězení druhé analytické otázky
*👉 Jak se ceny modelového bytu změnily za poslední roky? A jak se změnily mzdy?*


**vývoj ceny za m² v různých částech Prahy po letech**
```sql
SELECT
    "MC_spravne",
    "YEAR",
    ROUND(AVG("data_price"/"plocha")) AS "prumerna_cena_na_m2"
FROM "vycistena_tabulka_oprava_2"
WHERE 
        "data_price" IS NOT NULL
    AND "data_price" != ''
    AND "plocha" IS NOT NULL
    AND "plocha" != ''
    AND "MC_spravne" IS NOT NULL
GROUP BY "MC_spravne", "YEAR"
ORDER BY 1, 2 DESC;
```


**vývoj ceny za m² v různých částech Prahy - po letech a kvartálech**
```sql
SELECT
    "MC_spravne" AS MC,
    "YEAR" || '_' || "QUARTER" AS ROKKVARTAL,
    ROUND(AVG("data_price"/"plocha")) AS PRUMERNACENA
FROM "vycistena_tabulka_oprava_2"
WHERE 
        "data_price" IS NOT NULL
    AND "data_price" != ''
    AND "plocha" IS NOT NULL
    AND "plocha" != ''
    AND "MC_spravne" IS NOT NULL
GROUP BY 1, 2
ORDER BY 1, 2 DESC;
```


**najoinování dat o mzdách na hlavní dataset realit**
```sql
CREATE OR REPLACE TABLE "reality_mzdy" AS
SELECT * EXCLUDE "ROK"
FROM "vycistena_tabulka_oprava_2"
LEFT JOIN "Data_mzdy" ON "Data_mzdy"."ROK" = "vycistena_tabulka_oprava_2"."YEAR";
```

**najoinování dat o inflaci na dataset reality_mzdy**
```sql
CREATE OR REPLACE TABLE "reality_mzdy_inflace" AS
SELECT * EXCLUDE "rok"
FROM "reality_mzdy"
LEFT JOIN "Data_inflace" ON "Data_inflace"."rok" = "reality_mzdy"."YEAR";
```

**porovnání vývoje průměrné ceny na m², mediánových mezd a inflace v čase**
```sql
SELECT 
    "YEAR" AS "ROK",
    ROUND(AVG("data_price"/"plocha")) AS "PRUMERNA_CENA",
    AVG("MEDIAN_CELKEM") AS "MEDIANOVA_MZDA",
    AVG("inflace") AS "INFLACE"
FROM "reality_mzdy_inflace"
WHERE 
        "data_price" IS NOT NULL
    AND "data_price" != ''
    AND "plocha" IS NOT NULL
    AND "plocha" != ''
GROUP BY 1
ORDER BY 1;
```